In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold, cross_validate
import pandas as pd

In [2]:
#Load the Data
data=pd.read_csv('data.csv')
x=data.drop('REPEAT',axis=1)
y=data.REPEAT

In [3]:
#Create Model
model1=DecisionTreeClassifier(max_depth=8,min_samples_leaf=100,min_impurity_decrease=0,ccp_alpha=0.00008)
model2=KNeighborsClassifier(n_neighbors=8,leaf_size=20,n_jobs=-1,metric='cityblock')
model3 = MLPClassifier(hidden_layer_sizes=(90,),max_iter=1000,learning_rate='adaptive',warm_start=True)
model = VotingClassifier(estimators=[('tree', model1), ('knn', model2), ('mlp', model3)], voting='soft',n_jobs=-1)

In [4]:
#Perform Cross-Validation with K-Fold
kf=KFold(n_splits=5,shuffle=True, random_state=42)
result=cross_validate(model,x,y,cv=kf,scoring=('accuracy','f1_weighted'),return_train_score=bool,return_estimator=bool)

In [5]:
#get the best model
best_model=result['estimator'][result['test_f1_weighted'].argmax()]
#predivt the whole data
y_pred=best_model.predict(x)
#save as a csv file
pd.DataFrame({'y_pred':y_pred}).to_csv('y_pred.csv',index=True)

In [6]:
#print the accuracy and f1-score of test and training data
result_pd=pd.DataFrame(
    {'test':[result['test_accuracy'].mean(),result['test_f1_weighted'].mean()],
     'train':[result['train_accuracy'].mean(),result['train_f1_weighted'].mean()]
     }
    )
result_pd.index=['accuracy','f1-score']
result_pd

,test,train
accuracy,0.820103,0.836017
f1-score,0.800404,0.818413
